#Setup

In [0]:
from __future__ import absolute_import, print_function, division
from google.colab import files
from collections import Counter, defaultdict
from copy import deepcopy
from IPython.display import clear_output
from random import randint

import json
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf

CHECKPOINT_DIR = './checkpoints/' 
TEXT_ENCODING = 'utf-8'



class RNN(object):
  """Represents a Recurrent Neural Network using LSTM cells.

  Attributes:
    num_layers: The integer number of hidden layers in the RNN.
    state_size: The size of the state in each LSTM cell.
    num_classes: Number of output classes. (E.g. 256 for Extended ASCII).
    batch_size: The number of training sequences to process per step.
    sequence_length: The number of chars in a training sequence.
    batch_index: Index within the dataset to start the next batch at.
    on_gpu_sequences: Generates the training inputs for a single batch.
    on_gpu_targets: Generates the training labels for a single batch.
    input_symbol: Placeholder for a single label for use during inference.
    temperature: Used when sampling outputs. A higher temperature will yield
      more variance; a lower one will produce the most likely outputs. Value
      should be between 0 and 1.
    initial_state: The LSTM State Tuple to initialize the network with. This
      will need to be set to the new_state computed by the network each cycle.
    logits: Unnormalized probability distribution for the next predicted
      label, for each timestep in each sequence.
    output_labels: A [batch_size, 1] int32 tensor containing a predicted
      label for each sequence in a batch. Only generated in infer mode.
  """
  def __init__(self,
               rnn_num_layers=1,
               rnn_state_size=128,
               num_classes=256,
               rnn_batch_size=1,
               rnn_sequence_length=1):
    self.num_layers = rnn_num_layers
    self.state_size = rnn_state_size
    self.num_classes = num_classes
    self.batch_size = rnn_batch_size
    self.sequence_length = rnn_sequence_length
    self.batch_shape = (self.batch_size, self.sequence_length)
    print("Built LSTM: ",
          self.num_layers ,self.state_size ,self.num_classes ,
          self.batch_size ,self.sequence_length ,self.batch_shape)


  def build_training_model(self, dropout_rate, data_to_load):
    """Sets up an RNN model for running a training job.

    Args:
      dropout_rate: The rate at which weights may be forgotten during training.
      data_to_load: A numpy array of containing the training data, with each
        element in data_to_load being an integer representing a label. For
        example, for Extended ASCII, values may be 0 through 255.

    Raises:
      ValueError: If mode is data_to_load is None.
    """
    if data_to_load is None:
      raise ValueError('To continue, you must upload training data.')
    inputs = self._set_up_training_inputs(data_to_load)
    self._build_rnn(inputs, dropout_rate)

  def build_inference_model(self):
    """Sets up an RNN model for generating a sequence element by element.
    """
    self.input_symbol = tf.placeholder(shape=[1, 1], dtype=tf.int32)
    self.temperature = tf.placeholder(shape=(), dtype=tf.float32,
                                      name='temperature')
    self.num_options = tf.placeholder(shape=(), dtype=tf.int32,
                                      name='num_options')
    self._build_rnn(self.input_symbol, 0.0)

    self.temperature_modified_logits = tf.squeeze(
        self.logits, 0) / self.temperature

    #for beam search
    self.normalized_probs = tf.nn.softmax(self.logits)

    self.output_labels = tf.multinomial(self.temperature_modified_logits,
                                        self.num_options)

  def _set_up_training_inputs(self, data):
    self.batch_index = tf.placeholder(shape=(), dtype=tf.int32)
    batch_input_length = self.batch_size * self.sequence_length

    input_window = tf.slice(tf.constant(data, dtype=tf.int32),
                            [self.batch_index],
                            [batch_input_length + 1])

    self.on_gpu_sequences = tf.reshape(
        tf.slice(input_window, [0], [batch_input_length]), self.batch_shape)

    self.on_gpu_targets = tf.reshape(
        tf.slice(input_window, [1], [batch_input_length]), self.batch_shape)

    return self.on_gpu_sequences

  def _build_rnn(self, inputs, dropout_rate):
    """Generates an RNN model using the passed functions.

    Args:
      inputs: int32 Tensor with shape [batch_size, sequence_length] containing
        input labels.
      dropout_rate: A floating point value determining the chance that a weight
        is forgotten during evaluation.
    """
    # Alias some commonly used functions
    dropout_wrapper = tf.contrib.rnn.DropoutWrapper
    lstm_cell = tf.contrib.rnn.LSTMCell
    multi_rnn_cell = tf.contrib.rnn.MultiRNNCell

    self._cell = multi_rnn_cell(
        [dropout_wrapper(lstm_cell(self.state_size), 1.0, 1.0 - dropout_rate)
         for _ in range(self.num_layers)])

    self.initial_state = self._cell.zero_state(self.batch_size, tf.float32)

    embedding = tf.get_variable('embedding',
                                [self.num_classes, self.state_size])

    embedding_input = tf.nn.embedding_lookup(embedding, inputs)
    output, self.new_state = tf.nn.dynamic_rnn(self._cell, embedding_input,
                                               initial_state=self.initial_state)

    self.logits = tf.contrib.layers.fully_connected(output, self.num_classes,
                                                    activation_fn=None)

## Get the training data.

We can either download the works of Shakespeare to train on or upload a plain text file (in the next section).

In [0]:
shakespeare_url = "https://ocw.mit.edu/ans7870/6/6.006/s08/lecturenotes/files/t8.shakespeare.txt"
import urllib
file_contents = urllib.urlopen(shakespeare_url).read()
file_name = "shakespeare"
file_contents = file_contents[10501:]  # Skip headers and start at content
print("An excerpt: \n", file_contents[:664])

##Optional: if you want to train on your own training file, run the next cell. <p>Otherwise press 'cancel upload' to skip (or just don't run the following code cell).

Any file you choose to upload must be plain text, not PDF, Microsoft Word, etc.

In [0]:
uploaded = files.upload()

if uploaded:
  if type(uploaded) is not dict: uploaded = uploaded.files  ## Deal with filedit versions
  file_bytes = uploaded[uploaded.keys()[0]]
  utf8_string = file_bytes.decode(TEXT_ENCODING)
  file_contents = utf8_string if files else ''
  file_name = uploaded.keys()[0]
  print("An excerpt from training data: \n", file_contents[:664])


# Create the LSTM 


In [0]:
class RNN(object):
  """Represents a Recurrent Neural Network using LSTM cells.

  Attributes:
    num_layers: The integer number of hidden layers in the RNN.
    state_size: The size of the state in each LSTM cell.
    num_classes: Number of output classes. (E.g. 256 for Extended ASCII).
    batch_size: The number of training sequences to process per step.
    sequence_length: The number of chars in a training sequence.
    batch_index: Index within the dataset to start the next batch at.
    on_gpu_sequences: Generates the training inputs for a single batch.
    on_gpu_targets: Generates the training labels for a single batch.
    input_symbol: Placeholder for a single label for use during inference.
    temperature: Used when sampling outputs. A higher temperature will yield
      more variance; a lower one will produce the most likely outputs. Value
      should be between 0 and 1.
    initial_state: The LSTM State Tuple to initialize the network with. This
      will need to be set to the new_state computed by the network each cycle.
    logits: Unnormalized probability distribution for the next predicted
      label, for each timestep in each sequence.
    output_labels: A [batch_size, 1] int32 tensor containing a predicted
      label for each sequence in a batch. Only generated in infer mode.
  """
  def __init__(self,
               rnn_num_layers=1,
               rnn_state_size=128,
               num_classes=256,
               rnn_batch_size=1,
               rnn_sequence_length=1):
    self.num_layers = rnn_num_layers
    self.state_size = rnn_state_size
    self.num_classes = num_classes
    self.batch_size = rnn_batch_size
    self.sequence_length = rnn_sequence_length
    self.batch_shape = (self.batch_size, self.sequence_length)
    print("Built LSTM: ",
          self.num_layers ,self.state_size ,self.num_classes ,
          self.batch_size ,self.sequence_length ,self.batch_shape)


  def build_training_model(self, dropout_rate, data_to_load):
    """Sets up an RNN model for running a training job.

    Args:
      dropout_rate: The rate at which weights may be forgotten during training.
      data_to_load: A numpy array of containing the training data, with each
        element in data_to_load being an integer representing a label. For
        example, for Extended ASCII, values may be 0 through 255.

    Raises:
      ValueError: If mode is data_to_load is None.
    """
    if data_to_load is None:
      raise ValueError('To continue, you must upload training data.')
    inputs = self._set_up_training_inputs(data_to_load)
    self._build_rnn(inputs, dropout_rate)

  def build_inference_model(self):
    """Sets up an RNN model for generating a sequence element by element.
    """
    self.input_symbol = tf.placeholder(shape=[1, 1], dtype=tf.int32)
    self.temperature = tf.placeholder(shape=(), dtype=tf.float32,
                                      name='temperature')
    self.num_options = tf.placeholder(shape=(), dtype=tf.int32,
                                      name='num_options')
    self._build_rnn(self.input_symbol, 0.0)

    self.temperature_modified_logits = tf.squeeze(
        self.logits, 0) / self.temperature

    #for beam search
    self.normalized_probs = tf.nn.softmax(self.logits)

    self.output_labels = tf.multinomial(self.temperature_modified_logits,
                                        self.num_options)

  def _set_up_training_inputs(self, data):
    self.batch_index = tf.placeholder(shape=(), dtype=tf.int32)
    batch_input_length = self.batch_size * self.sequence_length

    input_window = tf.slice(tf.constant(data, dtype=tf.int32),
                            [self.batch_index],
                            [batch_input_length + 1])

    self.on_gpu_sequences = tf.reshape(
        tf.slice(input_window, [0], [batch_input_length]), self.batch_shape)

    self.on_gpu_targets = tf.reshape(
        tf.slice(input_window, [1], [batch_input_length]), self.batch_shape)

    return self.on_gpu_sequences

  def _build_rnn(self, inputs, dropout_rate):
    """Generates an RNN model using the passed functions.

    Args:
      inputs: int32 Tensor with shape [batch_size, sequence_length] containing
        input labels.
      dropout_rate: A floating point value determining the chance that a weight
        is forgotten during evaluation.
    """
    # Alias some commonly used functions
    dropout_wrapper = tf.contrib.rnn.DropoutWrapper
    lstm_cell = tf.contrib.rnn.LSTMCell
    multi_rnn_cell = tf.contrib.rnn.MultiRNNCell

    self._cell = multi_rnn_cell(
        [dropout_wrapper(lstm_cell(self.state_size), 1.0, 1.0 - dropout_rate)
         for _ in range(self.num_layers)])

    self.initial_state = self._cell.zero_state(self.batch_size, tf.float32)

    embedding = tf.get_variable('embedding',
                                [self.num_classes, self.state_size])

    embedding_input = tf.nn.embedding_lookup(embedding, inputs)
    output, self.new_state = tf.nn.dynamic_rnn(self._cell, embedding_input,
                                               initial_state=self.initial_state)

    self.logits = tf.contrib.layers.fully_connected(output, self.num_classes,
                                                    activation_fn=None)


### Define LSTM training parameters.
You can leave these default values and just run this code cell as is. Later, you can come back here and experiment wth these values. 
These parameters are just for training. Further down at the inference step, we'll define parameters for the text-generation step.

In [0]:
num_layers = 2
learning_rate = 0.001
state_size = 256
batch_size = 64
sequence_length = 256
num_training_steps = 30000 # Takes about 40 minuets 
steps_per_epoch = 500
learning_rate_decay = 0.95
gradient_clipping = 5.0

def get_loss(logits, targets, target_weights):
  with tf.name_scope('loss'):
    return tf.contrib.seq2seq.sequence_loss(
        logits,
        targets,
        target_weights,
        average_across_timesteps=True)

def get_optimizer(loss, initial_learning_rate, gradient_clipping, global_step,
                  decay_steps, decay_rate):

  with tf.name_scope('optimizer'):
    computed_learning_rate = tf.train.exponential_decay(
        initial_learning_rate,
        global_step,
        decay_steps,
        decay_rate,
        staircase=True)

    optimizer = tf.train.AdamOptimizer(computed_learning_rate)
    trained_vars = tf.trainable_variables()
    gradients, _ = tf.clip_by_global_norm(
        tf.gradients(loss, trained_vars),
        gradient_clipping)
    training_op = optimizer.apply_gradients(
        zip(gradients, trained_vars),
        global_step=global_step)

    return training_op, computed_learning_rate


class LossPlotter(object):
  def __init__(self, history_length):
    self.global_steps = []
    self.losses = []
    self.averaged_loss_x = []
    self.averaged_loss_y = []
    self.history_length = history_length

  def draw_plots(self):
    self._update_averages(self.global_steps, self.losses,
                          self.averaged_loss_x, self.averaged_loss_y)

    plt.title('Average Loss Over Time')
    plt.xlabel('Global Step')
    plt.ylabel('Loss')
    plt.plot(self.averaged_loss_x, self.averaged_loss_y, label='Loss/Time (Avg)')
    plt.plot()
    plt.plot(self.global_steps, self.losses,
             label='Loss/Time (Last %d)' % self.history_length,
             alpha=.1, color='r')
    plt.plot()
    plt.legend()
    plt.show()

    plt.title('Loss for the last 100 Steps')
    plt.xlabel('Global Step')
    plt.ylabel('Loss')
    plt.plot(self.global_steps, self.losses,
             label='Loss/Time (Last %d)' % self.history_length, color='r')
    plt.plot()
    plt.legend()
    plt.show()

    # The notebook will be slowed down at the end of training if we plot the
    # entire history of raw data. Plot only the last 100 steps of raw data,
    # and the average of each 100 batches. Don't keep unused data.
    self.global_steps = []
    self.losses = []
    self.learning_rates = []

  def log_step(self, global_step, loss):
    self.global_steps.append(global_step)
    self.losses.append(loss)

  def _update_averages(self, x_list, y_list,
                       averaged_data_x, averaged_data_y):
    averaged_data_x.append(x_list[-1])
    averaged_data_y.append(sum(y_list) / self.history_length)

## Prepare the LSTM



Convert the plain text file into arrays of tokens (and, later,  back).


In [0]:
import string

class TokenMapper(object):
  
  def __init__(self):
    self.token_mapping = {}
    self.reverse_token_mapping = {}
    
  def buildFromData(self, utf8_string, limit=0.00004):
    print("Build token dictionary.")
    total_num = len(utf8_string)
    sorted_tokens = sorted(Counter(utf8_string.decode('utf8')).items(), 
                           key=lambda x: -x[1])
    # Filter tokens: Only allow printable characters (not control chars) and
    # limit to ones that are resonably common, i.e. skip strange esoteric 
    # characters in order to reduce the dictionary size.
    filtered_tokens = filter(lambda t: t[0] in string.printable or 
                             float(t[1])/total_num > limit, sorted_tokens)
    tokens, counts = zip(*filtered_tokens)
    self.token_mapping = dict(zip(tokens, range(len(tokens))))
    for c in string.printable:
      if c not in self.token_mapping:
        print("Skipped token for: ", c)
    self.reverse_token_mapping = {
        val: key for key, val in self.token_mapping.items()}
    print("Created dictionary: %d tokens"%len(self.token_mapping))
  
  def mapchar(self, char):
    if char in self.token_mapping:
      return self.token_mapping[char]
    else:
      return self.token_mapping[' ']
  
  def mapstring(self, utf8_string):
    return [self.mapchar(c) for c in utf8_string]
  
  def maptoken(self, token):
    return self.reverse_token_mapping[token]
  
  def maptokens(self, int_array):
    return ''.join([self.reverse_token_mapping[c] for c in int_array])
  
  def size(self):
    return len(self.token_mapping)
  

  def print(self):
    for k,v in sorted(self.token_mapping.items(),key=itemgetter(1)):
      print(k, v)
      
  def alphabet(self):
    return ''.join([k for k,v in sorted(self.token_mapping.items(),key=itemgetter(1) )])


  
  def save(self, path):
    with open(path, 'wb') as json_file:
      json.dump(self.token_mapping, json_file)
  
  def restore(self, path):
    with open(path, 'r') as json_file:
      self.token_mapping = {}
      self.token_mapping.update(json.load(json_file))
      self.reverse_token_mapping = {val: key for key, val in self.token_mapping.items()}
      
      
# Clean the checkpoint directory and make a fresh one
!rm -rf {CHECKPOINT_DIR}
!mkdir {CHECKPOINT_DIR}
!ls -lt

chars_in_batch = (sequence_length * batch_size)
file_len = len(file_contents)
unique_sequential_batches = file_len // chars_in_batch

mapper = TokenMapper()
mapper.buildFromData(file_contents)
mapper.save(''.join([CHECKPOINT_DIR, 'token_mapping.json']))

input_values = mapper.mapstring(file_contents)
print('Done.')

##Build the LSTM

In [0]:
tf.reset_default_graph()
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
print('Constructing model...')

model = RNN(
    rnn_num_layers=num_layers,
    rnn_state_size=state_size,
    num_classes=mapper.size(),
    rnn_batch_size=batch_size,
    rnn_sequence_length=sequence_length)

model.build_training_model(0.05, np.asarray(input_values))
print('Constructed model successfully.')

print('Setting up training session...')
neutral_target_weights = tf.constant(
    np.ones(model.batch_shape),
    tf.float32
)
loss = get_loss(model.logits, model.on_gpu_targets, neutral_target_weights)
global_step = tf.get_variable('global_step', shape=(), trainable=False,
                              dtype=tf.int32)
training_step, computed_learning_rate = get_optimizer(
    loss,
    learning_rate,
    gradient_clipping,
    global_step,
    steps_per_epoch,
    learning_rate_decay
)


# Create a supervisor that will checkpoint the model in the CHECKPOINT_DIR
sv = tf.train.Supervisor(
    logdir=CHECKPOINT_DIR,
    global_step=global_step,
    save_model_secs=30)
print()
print('Ready to train.')

Begin / resume the training cycle. 
This could take a while. Every so often it saves a checkpoint so we don't lose our progress. To monitor the progress of training, feel free to stop the training every so often and run the inference cell to generate text with your model.
<p>
This code cell will attempt to pick up training where it left off, if a previous checkpoint exists.<p>
  This cell can be stopped if you like so you can generate text with the LSTM in the next section.

In [0]:
from datetime import datetime
start_time = datetime.now()

with sv.managed_session(config=config) as sess:
  print('Training supervisor successfully initialized all variables.')
  if not file_len:
    raise ValueError('To continue, you must upload training data.')
  elif file_len < chars_in_batch:
    raise ValueError('To continue, you must upload a larger set of data.')

  plotter = LossPlotter(100)
  step_number = sess.run(global_step)
  zero_state = sess.run([model.initial_state])
  max_batch_index = (unique_sequential_batches - 1) * chars_in_batch
  while not sv.should_stop() and step_number < num_training_steps:
    feed_dict = {
        model.batch_index: randint(0, max_batch_index),
        model.initial_state: zero_state
        }
    [_, _, training_loss, step_number, current_learning_rate, _] = sess.run(
        [model.on_gpu_sequences,
         model.on_gpu_targets,
         loss,
         global_step,
         computed_learning_rate,
         training_step],
        feed_dict)
    plotter.log_step(step_number, training_loss)
    if step_number % 100 == 0:
#       clear_output(True)
      plotter.draw_plots()
      print('Latest checkpoint is: %s' %
            tf.train.latest_checkpoint(CHECKPOINT_DIR))
      print('Learning Rate is: %f' %
            current_learning_rate)

    if step_number % 10 == 0:
      print('global step %d, loss=%f' % (step_number, training_loss))

# clear_output(True)

print('Training completed in HH:MM:SS = ', datetime.now()-start_time)
print('Latest checkpoint is: %s' %
      tf.train.latest_checkpoint(CHECKPOINT_DIR))

# Generate text


Run the code cell below to have the LSTM automatically generate text.<p> Keep in mind the LSTM has no knowledge of text (or anything else) beyond the relatively small amount of text it was trained on.

Try experimenting with different values for 

*   start_of_generated_text
*   length_of_generated_text
*   creativity


In [0]:
start_of_generated_text = " Alas"
length_of_generated_text = 2000
creativity = 0.85  # between 0.0 and 1.0


class BeamSearchCandidate(object):
  """Represents a node within the search space during Beam Search.

  Attributes:
    state: The resulting RNN state after the given sequence has been generated.
    sequence: The sequence of selections leading to this node.
    probability: The probability of the sequence occurring, computed as the sum
      of the probabilty of each character in the sequence at its respective
      step.
  """

  def __init__(self, init_state, sequence, probability):
    self.state = init_state
    self.sequence = sequence
    self.probability = probability

  def search_from(self, tf_sess, rnn_model, temperature, num_options):
    """Expands the num_options most likely next elements in the sequence.

    Args:
      tf_sess: The Tensorflow session containing the rnn_model.
      rnn_model: The RNN to use to generate the next element in the sequence.
      temperature: Modifies the probabilities of each character, placing
        more emphasis on higher probabilities as the value approaches 0.
      num_options: How many potential next options to expand from this one.

    Returns: A list of BeamSearchCandidate objects descended from this node.
    """
    expanded_set = []
    feed = {rnn_model.input_symbol: np.array([[self.sequence[-1]]]),
            rnn_model.initial_state: self.state,
            rnn_model.temperature: temperature,
            rnn_model.num_options: num_options}
    [predictions, probabilities, new_state] = tf_sess.run(
        [rnn_model.output_labels,
         rnn_model.normalized_probs,
         rnn_model.new_state], feed)
    # Get the indices of the num_beams next picks
    picks = [predictions[0][x] for x in range(len(predictions[0]))]
    for new_char in picks:
      new_seq = deepcopy(self.sequence)
      new_seq.append(new_char)
      expanded_set.append(
          BeamSearchCandidate(new_state, new_seq,
                              probabilities[0][0][new_char] + self.probability))
    return expanded_set

  def __eq__(self, other):
    return self.sequence == other.sequence

  def __ne__(self, other):
    return not self.__eq__(other)

  def __hash__(self):
    return hash(self.sequence())
  
  
  
  
def beam_search_generate_sequence(tf_sess, rnn_model, primer, temperature=0.85,
                                  termination_condition=None, num_beams=5):
  """Implements a sequence generator using Beam Search.

  Args:
    tf_sess: The Tensorflow session containing the rnn_model.
    rnn_model: The RNN to use to generate the next element in the sequence.
    temperature: Controls how 'Creative' the generated sequence is. Values
      close to 0 tend to generate the most likely sequence, while values
      closer to 1 generate more original sequences. Acceptable values are
      within (0, 1].
    termination_condition: A function taking one parameter, a list of
      integers, that returns True when a condition is met that signals to the
      RNN to return what it has generated so far.
    num_beams: The number of possible sequences to keep at each step of the
      generation process.

  Returns: A list of at most num_beams BeamSearchCandidate objects.
  """
  candidates = []

  rnn_current_state = sess.run([rnn_model.initial_state])
  #Initialize the state for the primer
  for primer_val in primer[:-1]:
    feed = {rnn_model.input_symbol: np.array([[primer_val]]),
            rnn_model.initial_state: rnn_current_state
           }
    [rnn_current_state] = tf_sess.run([rnn_model.new_state], feed)

  candidates.append(BeamSearchCandidate(rnn_current_state, primer, num_beams))

  while True not in [termination_condition(x.sequence) for x in candidates]:
    new_candidates = []
    for candidate in candidates:
      expanded_candidates = candidate.search_from(
          tf_sess, rnn_model, temperature, num_beams)
      for new in expanded_candidates:
        if new not in new_candidates:
          #do not reevaluate duplicates
          new_candidates.append(new)
    candidates = sorted(new_candidates,
                        key=lambda x: x.probability, reverse=True)[:num_beams]

  return [c for c in candidates if termination_condition(c.sequence)]


tf.reset_default_graph()
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.InteractiveSession(config=config)

model = RNN(
    rnn_num_layers=num_layers,
    rnn_state_size=state_size,
    num_classes=mapper.size(),
    rnn_batch_size=1,
    rnn_sequence_length=1)

model.build_inference_model()

sess.run(tf.global_variables_initializer())
saver = tf.train.Saver(tf.global_variables())
ckpt = tf.train.latest_checkpoint(CHECKPOINT_DIR)
saver.restore(sess, ckpt)

def gen(start_with, pred, creativity):
  int_array = mapper.mapstring(start_with)
  candidates = beam_search_generate_sequence(
      sess, model, int_array, temperature=creativity,
      termination_condition=pred,
      num_beams=1)
  gentext = mapper.maptokens(candidates[0].sequence)
  return gentext

def lengthlimit(n):
  return lambda text: len(text)>n
def sentences(n):
  return lambda text: mapper.maptokens(text).count(".")>=n
def paragraph():
  return lambda text: mapper.maptokens(text).count("\n")>0


print(gen(start_of_generated_text, lengthlimit(length_of_generated_text), creativity))

##Save a copy of our trained RNN for later.

In [0]:
save_model_to_drive = True  ## Set this to true to save directly to Google Drive.

def save_model_hyperparameters(path):
  with open(path, 'w')  as json_file:
    model_params = {
        'num_layers': model.num_layers,
        'state_size': model.state_size,
        'num_classes': model.num_classes
    }
    json.dump(model_params, json_file)

def save_to_drive(title, content):
  # Install the PyDrive wrapper & import libraries.
  !pip install -U -q PyDrive
  from pydrive.auth import GoogleAuth
  from pydrive.drive import GoogleDrive
  from google.colab import auth
  from oauth2client.client import GoogleCredentials

  # Authenticate and create the PyDrive client.
  auth.authenticate_user()
  gauth = GoogleAuth()
  gauth.credentials = GoogleCredentials.get_application_default()
  drive = GoogleDrive(gauth)

  newfile = drive.CreateFile({'title': title})
  newfile.SetContentFile(content)
  newfile.Upload()
  print('Uploaded file with ID %s as %s'% (newfile.get('id'),
         archive_name))
    
archive_name = ''.join([file_name,'_seedbank_char-rnn.zip'])
latest_model = tf.train.latest_checkpoint(CHECKPOINT_DIR).split('/')[2]
checkpoints_archive_path = ''.join(['./exports/',archive_name])
if not latest_model:
  raise ValueError('You must train a model before you can export one.')
  
%system mkdir exports
%rm -f {checkpoints_archive_path}
mapper.save(''.join([CHECKPOINT_DIR, 'token_mapping.json']))
save_model_hyperparameters(''.join([CHECKPOINT_DIR, 'model_attributes.json']))
%system zip '{checkpoints_archive_path}' -@ '{CHECKPOINT_DIR}checkpoint' \
            '{CHECKPOINT_DIR}token_mapping.json' \
            '{CHECKPOINT_DIR}model_attributes.json' \
            '{CHECKPOINT_DIR}{latest_model}.'*

if save_model_to_drive:
  save_to_drive(archive_name, checkpoints_archive_path)
else:
  files.download(checkpoints_archive_path)

  
print('Saving complete')

